In [17]:
import pandas as pd
import re
import numpy as np
## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
## nomalice parameters
from unidecode import unidecode

fileOut = "fuzzy_token_set_ratio_cert_boots.xlsx"

In [18]:
df = pd.read_csv("DF_BOOTCAMPS_2591.csv")

In [19]:
#CONSERVAR A TODOS AQUELLOS QUE NO TENGAN MISSING VALUES EN EDUCATION Y CERTIFICATION. 



dfc = df[df["certifications"].notna()] #DATAFRAME DE CERTIFICACIÓN
dfc = dfc.reset_index(drop = True) #REINICIAR INDICE

In [20]:
dfc.columns

Index(['id', 'scraping_services_id', 'linkedin_id', 'linkedin_profile_url',
       'last_name', 'first_name', 'v_extension', 'location', 'job_title',
       'company', 'job_history', 'snapshots', 'skills', 'education', 'about',
       'updated', 'middle_name', 'num_connections', 'interests', 'volunteered',
       'certifications', 'languages', 'recommendations', 'publications',
       'patents', 'industry', 'titles', 'company_name_complete',
       'job_title_complete'],
      dtype='object')

In [21]:
dfc.shape

(888, 29)

In [22]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'dev f\\.'
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'reprograma',
 'worldtech', 
 'bictia|bi.?tia', 
 'talento[ -]?digital']

# EXTRAER FORMACIÓN DESDE EL DATASET DE LOS NO-MISSING VALUES DE EDUCACIÓN

In [23]:
#CONVERTIR EN STRING
dfc["certifications"] =  dfc["certifications"].str.lower()

In [24]:
#CONVERTIR EN DICCIONARIOS LA EDUCACIÓN
L = []

for i in dfc["certifications"]:    
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        #print("ln", formacion)
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]

"{'certificate_authority': 'nepal engineering council', 'end': 'present', 'start': 'august 2014', 'title': 'certificate for the registered engineer'}"

In [25]:
K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO
regxCerAuth = "'certificate_authority': '.*?'"
regexTitle = "'title': '.*?'"

for individuo in L: #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'certificate_authority': '.*?'", ins)) ) > 0:
            c["certificate_authority"] =  re.findall("'certificate_authority': '.*?'", ins)[0].replace("'certificate_authority': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall(regexTitle, ins)) ) > 0:
            c["title"] =  re.findall("'title': '.*?'", ins)[0].replace("'title': ", "").replace("'", "")

        


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

{'certificate_authority': 'nepal engineering council',
 'title': 'certificate for the registered engineer',
 'start': 'august 2014',
 'end': 'present'}

In [26]:
#CREAR COLUMNA LIMPIA DE EDUCACIÓN
dfc["certifications"] = K

In [27]:
#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
dfc.certifications[0][0]

{'certificate_authority': 'nepal engineering council',
 'title': 'certificate for the registered engineer',
 'start': 'august 2014',
 'end': 'present'}

In [28]:
lista_boots = ['4Geeks',
 'Acámica',
 'Ada ITW',
 'Bedu Tech',
 'Bogotá Dev',
 'Bootcamp Institute',
 'Coderise International',
 'Desafío Latam',
 'Dev F.',
 'Digital House',
 'Generation Mexico',
 'Hack Academy',
 'HolaCode',
 'Holberton School',
 'iCreate Coding Academy',
 'IJALTI',
 'Iron Hack',
 'Jóvenes a Programar',
 'Keep Coding',
 'Kod1go',
 'Laboratoria',
 'Le Wagon',
 'Make it Real Camp',
 'MindHub ',
 'Muktek Academy',
 'Nivel Pro',
 'Noukod',
 'Plataforma 5',
 'Programa Valentina',
 'Pygmalion',
 'Reprograma',
 'World Tech Inc (a.k.a. World Tech Makers –WTM–)',
 'BICTIA',
 'Talento Digital']

In [29]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding|keep.?coding.io',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'pygmalion'
 'reprograma',
 'worldtech$',  #se cambió  para que solo termine con $
 'bictia|bi.?tia', 
 'talento[ -]?digital$'] #

 #VERDE: MÁS GRANDES DE LA, #AMARILLOS: MEDIANOS
 #EN PRINCIPIOS LOS VERDES SON LOS QUE DEBERIAN APARECER MÁS EN LOS PERFILES, EN PRINCIPIO.


 #BASE: LINKED-IN: + DIFICIL PARA HACER SCRAPING. EMPRESAS VENDEN DATOS (OBTUVIERON HACIENDO SCRAPING), PERFILES EGRESADOS DE BOOTCAMPS DE LISTA DE BOOTCAMPS EN LA. 3 RONDAS DE INTENTOS Y MANDAN MUESTRA PEQUEÑA. SUGIEREN MUCHOS DE LOS CAMBIOS.abs

 #NEGOCIACIÓN, PERFILES Y CUÁNTO CUESTA. MAX 8MIL. CADA FALSO POSITIVO ES UN PROBLEMA. 
 # LLEGABAN A MIL, PARECIA QUE FP NO ERAN PROBLEMA: MEJOR ESTRATEGIA PALABRAS GENERALES Y TRAEMOS 5MIL PERFILES Y DENTRO DE ESA DEBEN DE ESTAR TODOS ESTUDIANTES DE PERFILES PROFESIONALES DE BOOTCAMPS.
 # EGRESADOS EN LATIRNOAMERICA DEBERÍAN SER MUCHO MÁS DE 8MIL, EJ: DIGITAL HOUSE HABLAN DE 100MIL EGRESADOS

 #NO REPRESENTATIVIDAD: MUCHOS EGRESADOS POCA MUESTRA, NO TODOS TIENEN PERFIL DE LINKED-IN

 #PRIORIDAD

In [30]:
#LOOP PARA BUSCAR SI EL INDIVIDUO TIENE FORMACIÓN EN ALGÚN BOOTCAMP
c = []
for boot in range(len(boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfc.certifications)): #PARA CADA INDIVIDUO
        for h in range(len(dfc.certifications[i])): #DENTRO LA EDUCACIÓN DE CADA INDIVIDUO
            if "title" in dfc.certifications[i][h] and False: #SI TIENE EL CAMPO EDUCACIÓN
                bootCAMP = lista_boots[boot]
                bootCAMP = bootCAMP.lower()
                bootCAMP = unidecode(bootCAMP)
                norEdu =  unidecode(dfc.certifications[i][h]["title"])
                ## se hace un aproach a fuzzy
                wordLen = len(dfc.certifications[i][h]["title"])
                #ratio = fuzz.partial_ratio(bootCAMP,norEdu)
                ratio = fuzz.token_set_ratio(bootCAMP,norEdu)
                #print(ratio)
                ##dev f, minin string
                if ratio>85 and wordLen >=5:
                    c.append((dfc.id[i], lista_boots[boot], lista_boots[boot], dfc.certifications[i][h]['title'],ratio))
                    # c.append((dfe.id[i], h)) #AÑADE EL ID DE ESE INDIVIDUO Y EL LUGAR DONDE ESTÁ ESA EDUCACIÓN
                    print(f"{dfc['job_title'][i]}---{ratio}---{lista_boots[boot]}--{dfc.certifications[i][h]['title']} \n \n")




            elif "certificate_authority" in dfc.certifications[i][h]: #SI TIENE EL CAMPO EDUCACIÓN
                bootCAMP = lista_boots[boot]
                bootCAMP = bootCAMP.lower()
                bootCAMP = unidecode(bootCAMP)
                norEdu =  unidecode(dfc.certifications[i][h]["certificate_authority"])
                ## se hace un aproach a fuzzy
                wordLen = len(dfc.certifications[i][h]["certificate_authority"])
                #ratio = fuzz.partial_ratio(bootCAMP,norEdu)
                ratio = fuzz.token_set_ratio(bootCAMP,norEdu)
                #print(ratio)
                ##dev f, minin string
                if ratio>85 and wordLen >=5:
                    c.append((dfc.id[i], lista_boots[boot], lista_boots[boot], dfc.certifications[i][h]['certificate_authority'],ratio))
                    # c.append((dfe.id[i], h)) #AÑADE EL ID DE ESE INDIVIDUO Y EL LUGAR DONDE ESTÁ ESA EDUCACIÓN
                    print(f"{dfc['job_title'][i]}---{ratio}---{lista_boots[boot]}--{dfc.certifications[i][h]['certificate_authority']} \n \n")
                    


            #LOOP SIN INDICE
            # for edu in dfe.educacion[i]:
            # if "education" in edu:
            #     # print(boot, edu["education"])
            #     A = re.findall(boot, edu["education"])
            #     if len(A) > 0:
            #         c.append(A)
            #         m.append(boot)
            #         print(f" {dfe['job_title'][i]} -- {A} -- {boot} -- {edu['education']} \n \n")
            #         # print(f"{A} -- {boot} -- {edu['education']}")

⚡ Full Stack Developer JavaScript---100---Acámica--acamica 
 

⚡ Full Stack Developer JavaScript---100---Acámica--acamica 
 

⚡ Full Stack Developer JavaScript---100---Acámica--acamica 
 

⚡ Full Stack Developer JavaScript---100---Acámica--acamica 
 

Java Developer---100---Acámica--acamica 
 

Java Developer---100---Acámica--acamica 
 

Java Developer---100---Acámica--acamica 
 

Java Developer---100---Acámica--acamica 
 

Desarrollador de sistemas---100---Acámica--acamica 
 

Desarrollador de sistemas---100---Acámica--acamica 
 

Entrepreneur - Experience Manager---100---Acámica--acamica 
 

Entrepreneur - Experience Manager---100---Acámica--acamica 
 

Entrepreneur - Experience Manager---100---Acámica--acamica 
 

Entrepreneur - Experience Manager---100---Acámica--acamica 
 

Entrepreneur - Experience Manager---100---Acámica--acamica 
 

Entrepreneur - Experience Manager---100---Acámica--acamica 
 

Entrepreneur - Experience Manager---100---Acámica--acamica 
 

Entrepreneur - Experi

In [31]:
#CREAR DATAFRAME

c = list(set(c))
educ_boots = pd.DataFrame.from_records(c, columns=['id', 'regex_bootcamp', 'bootcamp', "match","ratio"])

In [32]:
#HACER MERGE CON EL RESTO DE COLUMNAS
educ_boots = pd.merge(educ_boots, dfc, "left", "id")
educ_boots.to_excel(fileOut, index = False)

In [33]:
df_educ 

NameError: name 'df_educ' is not defined